In [116]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', 500)

In [191]:
df = pd.read_csv("/Users/hansuho/Desktop/21-1 KNU/학교 수업/마케팅리서치/기말발표/폴더블_열이름변경/시트 1-폴더블 아이폰 6조.csv")
df.drop(columns='타임스탬프', inplace=True)
df = df.iloc[:, :-1]    # 휴대폰 번호 열 삭제
df.drop(index=12, inplace=True)    # 샤오미 홍미노트 행 삭제

In [192]:
# 7점 척도 데이터 범주 확인
print(df.iloc[:, 7].unique())
print(df.iloc[:, 12].unique())

['7 (매우 그렇다.)' '4 (보통이다.)' '3' '6' '5' '2']
['7    (매우 그렇다)' '4    (보통이다)' '6' '5' '3' '1    (전혀 그렇지 않다)']


In [193]:
score_dict = {"7 (매우 그렇다.)": 7, "7    (매우 그렇다)": 7,
              "6":6,
              "5":5,
              "4 (보통이다.)":4, "4    (보통이다)":4,
              "3":3,
              "2":2,
              "1 (전혀 그렇지 않다.)":1, "1    (전혀 그렇지 않다)":1}

likert7_cols = df.iloc[:, 6:18].columns.tolist()

for col in likert7_cols:
    tmp = []
    for item in df[col]:
        tmp.append(score_dict[item])
    df[col] = tmp

    
# 애플: 0, 삼성: 1
df['Manufacturer'] = [0 if x == "애플" else 1 for x in df['Manufacturer']]    


# 아이폰 시리즈에 se, xs 포함
for idx, item in enumerate(df['phone_series']):    
    if item == 'apple 아이폰 xs': df.iloc[idx, 3] = "애플 시리즈 (아이폰 12, 11 시리즈 등)"
    elif item == '아이폰se2': df.iloc[idx, 3] = "애플 시리즈 (아이폰 12, 11 시리즈 등)"

In [194]:
df['Manufacturer'] = [0 if x == "애플" else 1 for x in df['Manufacturer']]    # 애플: 0, 삼성: 1

for idx, item in enumerate(df['phone_series']):    # 아이폰 시리즈에 se, xs 포함
    if item == 'apple 아이폰 xs': df.iloc[idx, 3] = "애플 시리즈 (아이폰 12, 11 시리즈 등)"
    elif item == '아이폰se2': df.iloc[idx, 3] = "애플 시리즈 (아이폰 12, 11 시리즈 등)"

In [195]:
category_dict = {"6개월 미만": 0, '애플 시리즈 (아이폰 12, 11 시리즈 등)': 0, '애플의 폴더블폰': 0, '남성': 0,
               "6개월 ~ 1년 미만": 1, '삼성 폴더블폰 (Z-Fold 시리즈, Z-Flip 시리즈 등)': 1, '매우 불만족': 1,
               '전혀 그렇지 않다.': 1, '구입할 의향이 전혀 없다.': 1, '삼성의 Z-플립': 1, '여성': 1,
                 
               "1년 ~ 1년 6개월 미만": 2, '삼성 갤럭시 시리즈 (S 시리즈, Note 시리즈 등)': 2, '불만족': 2, '그렇지 않다.': 2, '구입할 의향이 없다.': 2,
               "1년 6개월 ~ 2년 미만": 3, 'LG (윙, V, Q 시리즈 등)': 3, '보통': 3, '보통이다.':3,
               "2년 이상": 4, '만족': 4, '그렇다.': 4, '구입할 의향이 있다.': 4,
               '매우 만족': 5, '매우 그렇다.': 5, '구입할 의향이 매우 있다.': 5}

categorical_cols = ['using_period', 'proper_change_period', 'phone_series', 'brand_satisfying', 'fordable_intension',
                    'intension_135', 'intension_155', 'intension_175', 'appford_flip', 'gender']

for col in categorical_cols:
    tmp = []
    for item in df[col]:
        tmp.append(category_dict[item])
    df[col] =tmp

In [196]:
for col in categorical_cols:
    tmp = []
    for item in df[col]:
        tmp.append(category_dict[item])
    df[col] =tmp

In [202]:
# 디자인 특성 및 브랜드 충성도 평균 점수 산출
df['design_feature'] = round(((df['design_v1'] +df['design_v2'] + df['design_v3'] + df['design_v4'] + df['design_v5'] + df['design_v6']) / 6), 1)
df['brand_loyalty'] = round(((df['loyalty_v1'] +df['loyalty_v2'] + df['loyalty_v3'] + df['loyalty_v4'] + df['loyalty_v5'] + df['loyalty_v6']) / 6), 1)

# 디자인 특성, 브랜드 충성도 하위 질문 
df.drop(columns=['design_v1', 'design_v2', 'design_v3', 'design_v4', 'design_v5', 'design_v6',
                 'loyalty_v1', 'loyalty_v2', 'loyalty_v3', 'loyalty_v4', 'loyalty_v5', 'loyalty_v6'], inplace=True)

In [205]:
df

,Manufacturer,using_period,proper_change_period,phone_series,brand_satisfying,satisfying_factor,purchase_reason,same_brand_item,channels,online_channel,online_reason,offline_channel,offline_channel.1,fordable_intension,intension_135,intension_155,intension_175,appford_flip,gender,age,design_feature,brand_loyalty
0,0,1,3,0,5,사용 편의성,주변 IT 기기와의 호환성 (7-1번 질문으로 이동하십시오),4개,오프라인 (10번 질문으로 이동하십시오),NaN,NaN,이마트(일렉트로마트),편리성,5,5,4,2,0,0,20대,6.5,7.0
1,0,1,4,0,5,디자인,성능,NaN,온라인 (9번 질문으로 이동하십시오),중고나라,가격,NaN,NaN,3,1,1,1,0,0,20대,6.5,6.8
2,1,1,4,2,4,성능,기존 사용 브랜드이기 때문에,2개,온라인 (9번 질문으로 이동하십시오),중고나라,자급제 사용을 위해서,NaN,NaN,1,1,1,1,1,0,20대,3.8,4.0
3,1,1,4,2,4,가격,합리적인 가격,2개,오프라인 (10번 질문으로 이동하십시오),NaN,NaN,휴대폰 구매 대리점(할인점),편리성,1,1,1,1,1,0,20대,2.8,4.0
4,0,2,4,0,4,디자인,기존 사용 브랜드이기 때문에,NaN,오프라인 (10번 질문으로 이동하십시오),NaN,NaN,애플 스토어,제품 실물을 확인하기 위해서,3,2,2,1,0,0,20대,5.5,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1,3,4,2,5,디자인,성능,3개,오프라인 (10번 질문으로 이동하십시오),NaN,NaN,휴대폰 구매 대리점(할인점),제품 실물을 확인하기 위해서,1,1,1,1,1,1,20대,4.8,5.8
63,0,1,3,0,3,디자인,디자인,NaN,온라인 (9번 질문으로 이동하십시오),공식 홈페이지,편리성,NaN,NaN,3,2,2,2,1,0,20대,4.3,4.3
64,1,4,3,2,5,사용 편의성,기존 사용 브랜드이기 때문에,4개,오프라인 (10번 질문으로 이동하십시오),NaN,NaN,삼성 디지털프라자,제품 실물을 확인하기 위해서,1,1,1,1,1,0,20대,6.0,6.8
65,0,0,3,0,4,디자인,주변 IT 기기와의 호환성 (7-1번 질문으로 이동하십시오),3개,오프라인 (10번 질문으로 이동하십시오),NaN,NaN,휴대폰 구매 대리점(할인점),제품 실물을 확인하기 위해서,1,3,1,1,0,0,10대,6.0,7.0
